# Efficient Transformer Training with PyTorch and Hugging Face

## Introduction

Transformer models have reshaped Natural Language Processing (NLP) and are increasingly used for computer vision tasks. However, their massive size and computational demands pose challenges for efficient training. In this tutorial, we'll explore strategies for optimizing Transformer training using PyTorch and Hugging Face while ensuring our code runs smoothly in Google Colab.

Here we use the following **efficiency techniques**:

* Mixed Precision Training: Automatic mixed precision (FP16) accelerates computations and reduces memory usage without heavily compromising accuracy.
* Gradient Scaling: Prevents numerical issues (underflow/overflow) commonly arising in mixed-:precision training.
* GPU Utilization: The code assumes GPU availability (device, torch.cuda.amp), significantly boosting performance for Transformer training.

## Environment Setup

Let's start by setting up a Colab environment and installing packages:

In [1]:
!pip install -q transformers torch matplotlib datasets

Import necessary libraries and check versions:

In [2]:
import transformers
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset

print(f"Transformers version: {transformers.__version__}")
print(f"Torch version: {torch.__version__}")

Transformers version: 4.37.2
Torch version: 2.1.0+cu121


## Dataset Preparation: Text Classification Example

In [3]:
dataset = load_dataset('glue', 'sst2')  # Let's use the SST-2 sentiment classification task

# Preprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize(batch):
  return tokenizer(batch['sentence'], padding='max_length', truncation=True, max_length=512)

dataset = dataset.map(tokenize, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Model Definition

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Optimization Techniques

In [5]:
# **1. Optimizer: AdamW**
# * Employs the AdamW optimizer, a sophisticated optimization algorithm with weight decay for regularization.
# * Gathers all trainable weights and biases from the Transformer model using `model.parameters()`.
# * Uses a learning rate of 0.00005 (specified by `lr=5e-5`) to control the pace of weight updates.
optimizer = AdamW(model.parameters(), lr=5e-5)

# **2. Training and Validation Split**
# * Calculates the number of samples to include in the training set (90% of the 'train' subset of your dataset).
# Training and validation splits
train_size = int(0.9 * len(dataset['train']))
train_dataset, val_dataset = dataset['train'].train_test_split(train_size).values()

# **3. Custom Collate Function (`collate_fn`)**
# * Defines how to assemble a batch of individual samples, handling variable-length sequences in Transformer models.
def collate_fn(batch):
    """Custom collate function to create tensors and transfer to device"""
    input_ids = torch.tensor([example['input_ids'] for example in batch])
    attention_mask = torch.tensor([example['attention_mask'] for example in batch])
    labels = torch.tensor([example['label'] for example in batch])

    return {
        'input_ids': input_ids.to(device),
        'attention_mask': attention_mask.to(device),
        'labels': labels.to(device)
    }

# **4. Data Loaders**
# * Creates data loaders for efficient batching during training and validation.
# * Shuffles training data (`shuffle=True`) for better generalization.
# * Employs the `collate_fn` to prepare batches.
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=4, collate_fn=collate_fn)


# **5. Learning Rate Scheduler**
# * Implements a scheduler to decrease the learning rate linearly over time for stable training.
# * Configures the scheduler based on the total number of training steps.
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, # Default value
                                            num_training_steps=num_training_steps)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
print(dataset['train'][0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0, 'input_ids': [101, 4750, 1207, 3318, 5266, 1121, 1103, 22467, 2338, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## Training Loop with Efficiency

Here, we utilize mixed precision.

### Key Benefits of Mixed Precision

* Speed: FP16 computations are generally much faster on modern GPUs, leading to significant training acceleration.
* Memory: Using FP16 reduces memory footprint, allowing you to train larger models or use larger batch sizes.

### Important Considerations

* Hardware: Mixed-precision benefits are most pronounced on GPUs with Tensor Cores (NVIDIA architecture).
* Stability: Not all models are equally suited for FP16. If you encounter NaN (Not a Number) values or instability, adjust your code or consider a less aggressive mixed-precision approach.

In the following part of the code:


```
scaler = torch.cuda.amp.GradScaler()
```

We instantiate a GradScaler object from PyTorch's automatic mixed precision (AMP) package. The GradScaler is crucial for preventing underflow or overflow issues often arising when working with mixed precision.

Then we use:

```
with torch.cuda.amp.autocast():
```

This context manager enables automatic mixed-precision within its block. Here's how it works:
* Casting: It automatically converts model operations and inputs to half-precision (FP16), where possible, for computational speed gains.
* Gradient Scaling: During backward propagation, gradients are scaled up to prevent vanishing gradients that can occur in FP16 computations.

```
outputs = model(**batch)
```

The forward pass of your Transformer model is executed within the autocast() context. Suitable computations will benefit from the efficiency of FP16.

```
loss = outputs.loss
```

The loss calculation is likely performed in full-precision (FP32) for numerical stability.

To integrate the scaling during the backward pass:

```
scaler.scale(loss).backward()  
scaler.step(optimizer)
scaler.update()
```

In [10]:
from tqdm import tqdm  # Progress visualization

# WARNING: Turning this to true may take more GPU!
evaluate_in_loop = False

def evaluate(dataloader):
    model.eval()
    total_loss, total_correct = 0.0, 0

    with tqdm(dataloader, unit="batch") as teval:  # Progress visualization
        for batch in teval:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item() * batch['input_ids'].size(0)
            predictions = outputs.logits.argmax(dim=-1)
            total_correct += (predictions == batch['labels']).sum().item()

            # Update progress bar description
            teval.set_description(f"Evaluating: Loss {loss.item():.3f}")

    avg_loss = total_loss / len(dataloader.dataset)
    accuracy = total_correct / len(dataloader.dataset)

    return {'loss': avg_loss, 'accuracy': accuracy}

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        # Mixed precision
        scaler = torch.cuda.amp.GradScaler()
        with torch.cuda.amp.autocast():
            outputs = model(**batch)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()

    if evaluate_in_loop:
      # Evaluation
      results = evaluate(val_dataloader)
      print(f"Epoch {epoch+1}, Validation Loss: {results['loss']:.3f}, Validation Accuracy: {results['accuracy']:.3f}")

  0%|          | 0/1684 [00:00<?, ?it/s]


## Evaluation

In [11]:
model.eval()
val_acc = evaluate(val_dataloader)
print(f"Validation Accuracy: {val_acc}")

Evaluating: Loss 0.598:   0%|          | 0/15154 [00:00<?, ?batch/s]

Validation Accuracy: {'loss': 3.943576010961662e-05, 'accuracy': 4.949351634935823e-05}
